In [1]:
# System level imports
import sys
import os

# ##### SET SYS PATH TO WHERE THE CODE IS. #####
# Note: Putting our code first in the sys path will make sure it gets picked up first
deep_dive_base_dir='/Users/dan/Code/LLM/DeepDive/backend'
sys.path.insert(0, deep_dive_base_dir)

os.environ["DEEP_DIVE_WORKSPACE"] = "/Users/dan/Code/LLM/DeepDive/workspace/notebooks"
#os.environ["DEEP_DIVE_WORKSPACE"] = "/Users/dan/Code/LLM/DeepDive/workspace/deep-dive"

os.environ["LOG_LEVEL"] = "TRACE"

from app.utils.logging_utilities import setup_logging, get_log_file_path
logger = setup_logging()
log_file_path = get_log_file_path(logger)
print(f"Log file path: {log_file_path}")
        

Log file path: /Users/dan/Code/LLM/DeepDive/notebooks/backend.log


In [ ]:
from app.agents.prompts import Prompts

prompts = Prompts()
prompts.print_params()

In [ ]:
from app.agents.search_agent import SearchAgent
search_agent = SearchAgent()
result=["Bernie Sanders political career highlights 2025", "Bernie Sanders recent speeches and statements March 2025", "Public opinion on Bernie Sanders policies as of March 2025"]
search_agent._is_list_of_strings(result)

In [ ]:
from app.search_methods.internet_search import ddg_search
query="What is Agentic AI?"
search_results = ddg_search(query, max_results=5, retries=3)
search_results

# Do Search

In [2]:
from app.agents.search_agent import SearchAgent
search_agent = SearchAgent()
query = "Who is Bernie Sanders?"
sub_queries = await search_agent._get_sub_queries(query)
sub_queries

2025-03-11 15:12:10 langchain_community.utils.user_agent WARNING: USER_AGENT environment variable not set, consider setting it to identify your requests.
2025-03-11 15:12:11 app INFO: Using provided workspace at: /Users/dan/Code/LLM/DeepDive/workspace/notebooks
2025-03-11 15:12:15 root DEBUG: PROMPT get_sub_queries response = ["Bernie Sanders political career highlights 2025", "Bernie Sanders recent speeches and statements March 2025", "Public opinion on Bernie Sanders policies as of March 2025"]


['Bernie Sanders political career highlights 2025',
 'Bernie Sanders recent speeches and statements March 2025',
 'Public opinion on Bernie Sanders policies as of March 2025']

In [3]:
sub_query_content = await search_agent._process_internet_query(sub_queries[0])
sub_query_content

2025-03-11 15:12:19 app DEBUG: scrape_urls extract_data_from_link link='https://apnews.com/article/bernie-sanders-democrats-trump-c213d5ae42737c956d46f6f7f17e5abd'
2025-03-11 15:12:19 app DEBUG: scrape_urls extract_data_from_link link='https://www.twincities.com/2025/03/10/democrats-bernie-sanders/'
2025-03-11 15:12:19 app DEBUG: scrape_urls extract_data_from_link link='https://abcnews.go.com/Politics/wireStory/drawing-huge-crowds-bernie-sanders-steps-leadership-anti-119623034'
2025-03-11 15:12:19 app DEBUG: scrape_urls extract_data_from_link link='https://www.vermontpublic.org/local-news/2025-03-10/drawing-huge-crowds-bernie-sanders-leadership-anti-trump-resistance'
2025-03-11 15:12:19 app DEBUG: scrape_urls extract_data_from_link link='https://otakukart.com/bernie-sanders-net-worth-2025-political-advocacy-books-and-wealth/'


'Source: https://otakukart.com/bernie-sanders-net-worth-2025-political-advocacy-books-and-wealth/\nTitle: \nContent: Net Worth  Bernie Sanders Net Worth 2025: Political Advocacy, Books, and Wealth\nFrom Grassroots Activist to Multi-Millionaire Senator—How Bernie Sanders Built His Fortune and Legacy  by\nOliver Johnson  February 13, 2025  in\nNet Worth  0  Bernie Sanders (Image via Getty)  Bernie Sanders is a well-known American politician with an estimated net worth of $3 million. He has served as a United States Senator for Vermont since 2007 and has run for president multiple times. His most recent financial disclosure in 2018 revealed a net worth of $1.7 million, largely earned through book royalties and advances. Over the years, Bernie has built wealth through various channels, and in 2018, his total income from book earnings and other sources reached $4.7 million.\n\nSource: https://otakukart.com/bernie-sanders-net-worth-2025-political-advocacy-books-and-wealth/\nTitle: \nContent:

In [4]:
import asyncio
context = await asyncio.gather(
    *[search_agent._process_internet_query(sub_query) for sub_query in sub_queries]
)
context

2025-03-11 15:12:34 app DEBUG: scrape_urls extract_data_from_link link='https://www.usnews.com/news/politics/articles/2025-03-10/drawing-huge-crowds-bernie-sanders-steps-into-leadership-of-the-anti-trump-resistance'
Error! : HTTPSConnectionPool(host='www.usnews.com', port=443): Read timed out. (read timeout=10)
2025-03-11 15:12:45 app DEBUG: scrape_urls extract_data_from_link link='https://www.newsweek.com/videos-bernie-sanders-large-midwest-rallies-take-off-social-media-2041843'
2025-03-11 15:12:45 app DEBUG: scrape_urls extract_data_from_link link='https://www.usatoday.com/story/news/2025/03/05/senator-bernie-sanders-president-donald-trump-joint-address/81570095007/'
2025-03-11 15:12:45 app DEBUG: scrape_urls extract_data_from_link link='https://www.sanders.senate.gov/press-releases/prepared-remarks-sanders-delivers-livestreamed-response-to-president-trumps-congressional-address/'
2025-03-11 15:12:45 app DEBUG: scrape_urls extract_data_from_link link='https://www.wpr.org/news/bernie-

['',
 'Source: https://www.sanders.senate.gov/press-releases/prepared-remarks-sanders-delivers-livestreamed-response-to-president-trumps-congressional-address/\nTitle: \nContent: Contact\nScheduling Requests\nPress Inquiries\nNewsletter Signup  Search  Search  PREPARED REMARKS: Sanders Delivers Livestreamed Response’ to President Trump’s Congressional Address\xa0\nHome » Media » Press Releases » PREPARED REMARKS: Sanders Delivers Livestreamed Response’ to President Trump’s Congressional Address  PREPARED REMARKS: Sanders Delivers Livestreamed Response’ to President Trump’s Congressional Address  March 4, 2025  WASHINGTON, March 4 – Sen. Bernie Sanders (I-Vt.) today delivered remarks following President Trump’s address to a joint session of Congress. Over 91,000 viewers watched in real time on his social media platforms.\nSanders’ remarks, as prepared for delivery, are below and can be watched\xa0here.\nHello everybody. Thanks for joining.\n\nSource: https://www.sanders.senate.gov/press

In [ ]:
from app.scrapers. scraper_methods import scrape_urls
urls = [entry['href'] for entry in search_results]
scrape_results = scrape_urls(urls)
scrape_results

In [ ]:
from app.embedding_methods.compressor import ContextCompressor
context_compressor = ContextCompressor( documents=scrape_results)
context_results = context_compressor.get_context(query, max_results=8)
context_results

In [ ]:
context_results

In [5]:
from app.agents.search_agent import SearchAgent
search_agent = SearchAgent()
query = "Who is Bernie Sanders?"
search_results = await search_agent.internet_search_context(query)
search_results


2025-03-11 15:46:26 root DEBUG: PROMPT get_sub_queries response = ["Bernie Sanders political career highlights 2025", "Bernie Sanders recent speeches and statements March 2025", "Public opinion on Bernie Sanders policies as of March 2025"]
2025-03-11 15:46:27 app DEBUG: scrape_urls extract_data_from_link link='https://abcnews.go.com/Politics/wireStory/drawing-huge-crowds-bernie-sanders-steps-leadership-anti-119623034'
2025-03-11 15:46:27 app DEBUG: scrape_urls extract_data_from_link link='https://apnews.com/article/bernie-sanders-democrats-trump-c213d5ae42737c956d46f6f7f17e5abd'
2025-03-11 15:46:27 app DEBUG: scrape_urls extract_data_from_link link='https://www.vermontpublic.org/local-news/2025-03-10/drawing-huge-crowds-bernie-sanders-leadership-anti-trump-resistance'
2025-03-11 15:46:27 app DEBUG: scrape_urls extract_data_from_link link='https://www.twincities.com/2025/03/10/democrats-bernie-sanders/'
2025-03-11 15:46:27 app DEBUG: scrape_urls extract_data_from_link link='https://www.

['Source: https://www.twincities.com/2025/03/10/democrats-bernie-sanders/\nTitle: \nContent: Show Caption1 of 5Sen. Bernie Sanders, I-Vt., speaks during a “Fighting Oligarchy: Where We Go From Here” event Saturday, March 8, 2025, at Lincoln High School in Warren, Mich. (AP Photo/Jose Juarez)\n\nSource: https://www.vermontpublic.org/local-news/2025-03-10/drawing-huge-crowds-bernie-sanders-leadership-anti-trump-resistance\nTitle: \nContent: APSen. Bernie Sanders speaks during a "Fighting Oligarchy: Where We Go From Here" event Saturday, March 8, at Lincoln High School in Warren, Mich. Sanders held all the events in swing U.S. House districts represented by Republicans.  Sanders, who was just elected to his fourth Senate term from Vermont, conceded that this is not the role he expected to play at this stage of his career.In fact, his team intentionally waited in the early weeks of the Trump presidency to launch what they are now calling his “stop oligarchy tour” to see if a high-profile D